Reference:

[**Introduction to LangChain for Agentic AI**](https://courses.analyticsvidhya.com/courses/take/introduction-to-langchain-for-agentic-ai/lessons/61748853-course-introduction) 

# Exploring Conversation Chains and Memory with LCEL

## Install OpenAI, and LangChain dependencies

In [ ]:
# !pip install -qq langchain
# !pip install -qq langchain-openai
# !pip install -qq langchain-community
# !pip install -qq langchain-chroma

## Enter Open AI API Key

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Working with LangChain Chains

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Also running on multiple data points can be done easily with chains.

Chain's are the legacy interface for "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

Here we will be using LCEL chains exclusively

### The Problem with Simple LLM Chains

Simple LLM Chains cannot keep a track of past conversation history

In [4]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

prompt_text = """{query}"""
prompt = ChatPromptTemplate.from_template(prompt_text)

llm_chain = prompt | chatgpt

response = llm_chain.invoke({"query": "What is the capital of France?"})
print(response.content)

The capital of France is Paris.


In [5]:
response = llm_chain.invoke({"query":"And what about India?"})
print(response.content)

Could you please provide more context or specify what aspect of India you are interested in? India is a vast country with a rich history, diverse culture, and significant developments in various fields such as politics, economy, technology, and social issues. Let me know what specific information you are looking for!


## Conversation Chains with LCEL

LangChain Expression Language (LCEL) connects prompts, models, parsers and retrieval components using a `|` pipe operator.

A conversation chain basically consists of user prompts, historical conversation memory and the LLM. The LLM uses the history memory to give more contextual answers to every new prompt or user query.

Memory is very important for having a true conversation with LLMs. LangChain allows us to manage conversation memory using various constructs. The main ones we will cover include:

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationSummaryMemory
- VectorStoreRetrieverMemory
- ChatMessageHistory
- SQLChatMessageHistory


These specialized memory classes implement different approaches to managing conversation context.They determines what information is retained and how it's presented to the language model

## Conversation Chains with ConversationBufferMemory

This is the simplest version of in-memory storage of historical conversation messages. It is basically a buffer for storing conversation memory.

Remember if you have a really long conversation, you might exceed the max token limit of the context window allowed for the LLM.

The `return_messages=True` parameter in `ConversationBufferMemory` controls the **format** in which conversation history is returned.

## What it does:

| `return_messages` | History Format | Use Case |
|---|---|---|
| `True` | List of message objects (`HumanMessage`, `AIMessage`) | Chat models (like `ChatOpenAI`) |
| `False` (default) | Single formatted string | Completion-style LLMs |

## Why it matters here:

In your code, you're using:

```14:14:2. Concepts/4. Chat Message Memory/2.2 Conversation_Chains_and_Memory_with_LCEL.ipynb
memory = ConversationBufferMemory(return_messages=True)
```

This is required because your prompt uses `MessagesPlaceholder`:

```9:9:2. Concepts/4. Chat Message Memory/2.2 Conversation_Chains_and_Memory_with_LCEL.ipynb
        MessagesPlaceholder(variable_name="history"), # This is where the conversation history will be stored
```

`MessagesPlaceholder` expects a **list of message objects**, not a string.

## Example comparison:

**With `return_messages=True`:**
```python
memory.load_memory_variables({})['history']
# Returns: [HumanMessage(content='Hello'), AIMessage(content='Hi there!')]
```

**With `return_messages=False`:**
```python
memory.load_memory_variables({})['history']
# Returns: "Human: Hello\nAI: Hi there!"
```

Since you're using `ChatOpenAI` (a chat model) with `ChatPromptTemplate`, you need the message objects format—hence `return_messages=True` is the correct choice.

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"), # This is where the conversation history will be stored
        ("human", "{query}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

In [8]:
print(type(memory))

<class 'langchain.memory.buffer.ConversationBufferMemory'>


In [9]:
# function to get historical conversation messages from the memory
print(memory.load_memory_variables({}))

{'history': []}


In [10]:
# Get buffer content directly
print(f"\nDirect buffer access:\n{memory.buffer}")


Direct buffer access:
[]


In [11]:
print(memory.load_memory_variables({})['history'])

[]


In [12]:
# lets create a function now which returns the list of messages from memory
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

print(get_memory_messages('What are the first four colors of a rainbow?'))

[]


In [13]:
type(RunnableLambda(get_memory_messages))

langchain_core.runnables.base.RunnableLambda

In [14]:
# testing out the function with a runnable lambda which will go into our chain
# this returns the history but we also need to send our current query to the prompt
print(RunnableLambda(get_memory_messages).invoke({'query': 'What are the first four colors of a rainbow?'}))

[]


In [15]:
# We use RunnablePassthrough.assign() to combine the current input with memory history
# This allows us to pass the query unchanged while adding the conversation history
# The .assign() method adds a new key 'history' to the input dictionary
# RunnableLambda(get_memory_messages) extracts the conversation history from memory
RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ).invoke({'query': 'What are the first four colors of a rainbow1?','query2':"sourav"})

{'query': 'What are the first four colors of a rainbow1?',
 'query2': 'sourav',
 'history': []}

In [16]:
chain1 = {
    'query': RunnableLambda(lambda x: x['query2'])
} |RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )
chain1.invoke({'query1': 'What are the first four colors of a rainbow1?','query2':"sourav"})

{'query': 'sourav', 'history': []}

In [17]:
chain2 = {
    'query': RunnableLambda(lambda x: x['query1'])
} |RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )

chain2.invoke({'query1': 'What are the first four colors of a rainbow1?','query2':"sourav"})

{'query': 'What are the first four colors of a rainbow1?', 'history': []}

In [18]:
runnable = RunnableLambda(lambda x: x['query1'])
runnable.invoke({'query1': 'What are the first four colors of a rainbow1?', 'query2': "sourav"})

'What are the first four colors of a rainbow1?'

In [19]:
def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = {
    'query': RunnableLambda(lambda x: x['query']), 
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    history=RunnableLambda(get_memory_messages)
)

chain.invoke({'query': 'What are the first four colors of a rainbow1?'})

{'query': 'What are the first four colors of a rainbow1?',
 'llm1': 'completion',
 'llm2': 'completion',
 'history': []}

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"), # This is where the conversation history will be stored
        ("human", "{query}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

query = {'query': 'What are the first four colors of a rainbow?'}
response = conversation_chain.invoke(query)
print(response)

content='The first four colors of a rainbow are red, orange, yellow, and green.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 30, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-CllE6wigfYOAsAdhyYmsZprFwaufu', 'finish_reason': 'stop', 'logprobs': None} id='run-5365c3bd-5541-4189-b1bb-70f41a97bb91-0' usage_metadata={'input_tokens': 30, 'output_tokens': 17, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [7]:
# response.dict() #deprecated
response.model_dump()

{'content': 'The first four colors of a rainbow are red, orange, yellow, and green.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 17,
   'prompt_tokens': 30,
   'total_tokens': 47,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_11f3029f6b',
  'id': 'chatcmpl-CllE6wigfYOAsAdhyYmsZprFwaufu',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-5365c3bd-5541-4189-b1bb-70f41a97bb91-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 30,
  'output_tokens': 17,
  'total_tokens': 47,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [8]:
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [9]:
print(memory.load_memory_variables({})['history'])

[]


In [10]:
query

{'query': 'What are the first four colors of a rainbow?'}

In [11]:
# Save the current conversation turn to memory.
# This ensures that the user's query and the model's response are stored,
# so that future queries can access the full conversation history.
# The `save_context` method takes the input query and the output response as arguments.
# Here, we save the query and the model's response content.
memory.save_context(query, {"output": response.content})

In [12]:
# Let's inspect the current conversation history in memory.
# This will show all previous exchanges (user queries and model responses) stored so far.
print(memory.load_memory_variables({})['history'])

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}), AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={})]


In [13]:
print("=== Messages in Chat Memory ===")
for i, message in enumerate(memory.chat_memory.messages):
    print(f"{i+1}. {message.type}: {message.content}")

=== Messages in Chat Memory ===
1. human: What are the first four colors of a rainbow?
2. ai: The first four colors of a rainbow are red, orange, yellow, and green.


In [14]:
# Get memory as variables (what would be passed to prompt)
memory_vars = memory.load_memory_variables({})
print(f"\nMemory as string:\n{memory_vars['history']}")


Memory as string:
[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}), AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={})]


In [15]:
query = {'query': 'and the other 3?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The other three colors of a rainbow are blue, indigo, and violet.


In [16]:
print(memory.load_memory_variables({})['history'])

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}), AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}), HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}), AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={})]


In [17]:
memory.load_memory_variables({})['history']

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={})]

In [18]:
print("=== Messages in Chat Memory ===")
for i, message in enumerate(memory.chat_memory.messages):
    print(f"{i+1}. {message.type}: {message.content}")

=== Messages in Chat Memory ===
1. human: What are the first four colors of a rainbow?
2. ai: The first four colors of a rainbow are red, orange, yellow, and green.
3. human: and the other 3?
4. ai: The other three colors of a rainbow are blue, indigo, and violet.


In [19]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).


In [21]:
print("=== Messages in Chat Memory ===")
for i, message in enumerate(memory.chat_memory.messages):
    print(f"{i+1}. {message.type}: {message.content}")

=== Messages in Chat Memory ===
1. human: What are the first four colors of a rainbow?
2. ai: The first four colors of a rainbow are red, orange, yellow, and green.
3. human: and the other 3?
4. ai: The other three colors of a rainbow are blue, indigo, and violet.
5. human: Explain AI in 2 bullet points
6. ai: - **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).


In [22]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.

- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous vehicles, and various other fields requiring advanced data analysis and pattern recognition.


In [23]:
print("=== Messages in Chat Memory ===")
for i, message in enumerate(memory.chat_memory.messages):
    print(f"{i+1}. {message.type}: {message.content}")

=== Messages in Chat Memory ===
1. human: What are the first four colors of a rainbow?
2. ai: The first four colors of a rainbow are red, orange, yellow, and green.
3. human: and the other 3?
4. ai: The other three colors of a rainbow are blue, indigo, and violet.
5. human: Explain AI in 2 bullet points
6. ai: - **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).
7. human: Now do the same for Deep Learning
8. ai: - **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layer

In [24]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

So far, we have discussed the following topics:

1. The first four colors of a rainbow (red, orange, yellow, green) and the remaining three (blue, indigo, violet).
2. A brief explanation of Artificial Intelligence (AI) in two bullet points.
3. A brief explanation of Deep Learning in two bullet points.


In [25]:
memory.load_memory_variables({})['history']

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations

In [26]:
print("=== Messages in Chat Memory ===")
for i, message in enumerate(memory.chat_memory.messages):
    print(f"{i+1}. {message.type}: {message.content}")

=== Messages in Chat Memory ===
1. human: What are the first four colors of a rainbow?
2. ai: The first four colors of a rainbow are red, orange, yellow, and green.
3. human: and the other 3?
4. ai: The other three colors of a rainbow are blue, indigo, and violet.
5. human: Explain AI in 2 bullet points
6. ai: - **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).
7. human: Now do the same for Deep Learning
8. ai: - **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layer

## Conversation Chains with ConversationBufferWindowMemory

If you have a really long conversation, you might exceed the max token limit of the context window allowed for the LLM when using `ConversationBufferMemory` so `ConversationBufferWindowMemory` helps in just storing the last K conversations (one conversation piece is one user message and the corresponding AI message from the LLM) and thus helps you manage token limits and costs

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# stores last 2 sets of human-AI conversations
memory = ConversationBufferWindowMemory(return_messages=True, k=3)

# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

In [30]:
query = {'query': 'What are the first four colors of a rainbow?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [31]:
query = {'query': 'and the other 3?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The other three colors of a rainbow are blue, indigo, and violet.


In [32]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).


In [33]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.

- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous vehicles, and various other fields requiring advanced data analysis and pattern recognition.


In [34]:
memory.load_memory_variables({})['history']

[HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Now do the same for Deep Learning', additional_kwargs={}

In [35]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

So far, we have discussed the following topics:

1. The colors of a rainbow, specifically mentioning blue, indigo, and violet.
2. A brief explanation of Artificial Intelligence (AI) in two bullet points.
3. A brief explanation of Deep Learning in two bullet points.


In [36]:
memory.load_memory_variables({})['history']

[HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Now do the same for Deep Learning', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, 

## Conversation Chains with ConversationSummaryMemory

If you have a really long conversation or a lot of messages, you might exceed the max token limit of the context window allowed for the LLM when using `ConversationBufferMemory`

`ConversationSummaryMemory` creates a summary of the conversation history over time. This can be useful for condensing information from the conversation messages over time.

This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [38]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        # MessagesPlaceholder(variable_name="history") - This placeholder inserts conversation history messages into the prompt
        # The variable_name can be any string - "history", "history_summary", "chat_history", etc.
        # It will be populated by the memory.load_memory_variables() function with the corresponding key
        MessagesPlaceholder(variable_name="history_summary"),
        ("human", "{query}"),
    ]
)

memory = ConversationSummaryMemory(return_messages=True, llm=chatgpt)
# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history_summary=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages as a summary to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

In [39]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, and problem-solving.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection), autonomous vehicles, customer service (chatbots), and more, enhancing efficiency and decision-making.


In [40]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- Deep learning is a subset of machine learning that uses neural networks with many layers to analyze and interpret complex data patterns, enabling tasks like image and speech recognition.
- It is widely applied in areas such as natural language processing, computer vision, and autonomous systems, significantly improving performance in these fields.


In [41]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks the AI to explain artificial intelligence in two bullet points. The AI defines artificial intelligence as the simulation of human intelligence processes by machines, enabling tasks like learning and problem-solving, and highlights its applications in fields such as healthcare, finance, autonomous vehicles, and customer service, which enhance efficiency and decision-making. The human then requests a similar explanation for deep learning, and the AI describes deep learning as a subset of machine learning that uses neural networks with many layers to analyze complex data patterns, enabling tasks like image and speech recognition, and notes its applications in natural language processing, computer vision, and autonomous systems, which significantly improve performance in these areas.', additional_kwargs={}, response_metadata={})]}

## Conversation Chains with VectorStoreRetrieverMemory

`VectorStoreRetrieverMemory` stores historical conversation messages in a vector store and queries the top-K most "relevant" history messages every time it is called.

This differs from most of the other Memory classes in that it doesn't explicitly track the order of interactions but retrieves history based on embedding similarity to the current question or prompt.

In this case, the "docs" are previous conversation snippets. This can be useful to refer to relevant pieces of information that the AI was told earlier in the conversation.

### Connect to  Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [43]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

### Create a Vector Database to store conversation history

Here we use the Chroma vector DB and initialize an empty database collection to store conversation messages

In [44]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='history_db',
                   embedding_function=openai_embed_model)

In [46]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import VectorStoreRetrieverMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# load 2 most similar conversation messages from vector db history for each new message \ prompt
# this uses cosine embedding similarity to load the top 2 similar messgages to the input prompt \ query
retriever = chroma_db.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 2})
memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)

# creating our conversation chain now
def get_memory_messages(query):
    return [memory.load_memory_variables(query)['history']]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

In [47]:
query = {'query': 'Tell me about AI'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, decision-making, and problem-solving, and is used in applications like virtual assistants, autonomous vehicles, and data analysis. AI continues to evolve, raising both opportunities and ethical considerations.


In [48]:
query = {'query': 'What about deep learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


Deep learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze and learn from large amounts of data. It excels in tasks such as image and speech recognition, natural language processing, and game playing. Deep learning models automatically extract features from raw data, making them highly effective for complex problems. However, they require significant computational power and large datasets for training.


In [49]:
query = {'query': 'Tell me about the fastest animal in the world in 2 lines'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The fastest animal in the world is the peregrine falcon, which can reach speeds of over 240 mph (386 km/h) during its hunting stoop (high-speed dive). On land, the cheetah holds the title, capable of sprinting up to 60-70 mph (97-113 km/h) in short bursts.


In [50]:
query = {'query': 'What about the cheetah?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The cheetah is the fastest land animal, capable of reaching speeds of 60-70 mph (97-113 km/h) in short bursts covering distances up to 1,500 feet (460 meters). Its unique adaptations, such as a lightweight body, long legs, and a flexible spine, enable it to accelerate rapidly and make sharp turns while chasing prey.



Now for a new query around machine learning even if the most recent conversation messages have been about animals, it uses the vector databases to load the last 2 historical conversations which are closest to the current question in terms of semantic similarity

In [51]:
print(memory.load_memory_variables({'query': 'What about machine learning?'})['history'])

query: What about deep learning
output: Deep learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze and learn from large amounts of data. It excels in tasks such as image and speech recognition, natural language processing, and game playing. Deep learning models automatically extract features from raw data, making them highly effective for complex problems. However, they require significant computational power and large datasets for training.
query: Tell me about AI
output: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, decision-making, and problem-solving, and is used in applications like virtual assistants, autonomous vehicles, and data analysis. AI continues to evolve, raising both opportunities 

In [52]:
query = {'query': 'What about machine learning?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Machine learning is a subset of artificial intelligence that focuses on the development of algorithms that enable computers to learn from and make predictions or decisions based on data. It involves training models on datasets to identify patterns and improve performance over time without being explicitly programmed. Machine learning is used in various applications, including recommendation systems, fraud detection, and predictive analytics. It can be categorized into supervised, unsupervised, and reinforcement learning.


In [53]:
print(memory.load_memory_variables({'query': 'What is the capital of India?'})['history'])

query: What about the cheetah?
output: The cheetah is the fastest land animal, capable of reaching speeds of 60-70 mph (97-113 km/h) in short bursts covering distances up to 1,500 feet (460 meters). Its unique adaptations, such as a lightweight body, long legs, and a flexible spine, enable it to accelerate rapidly and make sharp turns while chasing prey.
query: Tell me about AI
output: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, decision-making, and problem-solving, and is used in applications like virtual assistants, autonomous vehicles, and data analysis. AI continues to evolve, raising both opportunities and ethical considerations.
